## Coarse Graining Polymers using GRiTS

In [2]:
import grits
import mbuild as mb
import inspect
from polymer_dictionary import polymer_dictionary
from polymer_dictionary import polymer_fragments
from polymer_dictionary import element_dict
import gsd
import os
import glob
import string
from grits import CG_System
from grits import utils
import numpy as np

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/foyer/forcefield.py:34: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import iter_entry_points, resource_filename
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/pkg_resources/__init__.py:3146: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)



Support for writing out LAMMPS data files will be removed
in mbuild 1.0.
See GMSO (https://github.com/mosdef-hub/gmso/tree/main/gmso/formats/lammpsdata) for
continued support for LAMMPS.



In [ ]:
key_list = sorted(list(polymer_dictionary.keys())) # Aligning dictionary and path to aid automation
path = (os.getcwd()+"/") # Non-user specific path
molecule_list = sorted(glob.glob(path+"/mol2/"+"*.mol2"))
molecule_list.pop(10) # Removing polymer not tested
print(key_list)

In [ ]:
# Function to parse through GSD nomenclature and automatically get backbone segments
# TODO calculate COM instead of using SMILES parsing
def cg(filename):
    letters = string.ascii_uppercase
    name = os.path.basename(filename)
    name = (os.path.splitext(name)[0])[1:]
    new_gsd_name = "" + name
    
    final_cg = [polymer_fragments["P"]]
    grits_string = {}
    if name[0:4] == "CPDT":
        final_cg.append(polymer_fragments["CPDT"])
        name = name[4:]
    if name[0:3] == "IDT":
        final_cg.append(polymer_fragments["IDT"])
        name = name[3:]
    if name[0:3] == "FBT":
        final_cg.append(polymer_fragments["FBT"])
        name = name[3:]
    if name[0:2] == "PT":
        final_cg.append(polymer_fragments["PT"])
        name = name[2:]
    if name[0:2] == "_PT":
        final_cg.append(polymer_fragments["PT"])
        name = name[2:]
    if name[0:4] == "CPDT":
        final_cg.append(polymer_fragments["CPDT"])
        name = name[4:]
    if name[0:10] == "CPDT_eneHD":
        final_cg.append(polymer_fragments["CPDT_eneHD"])
        name = name[10:0]
    for i in range(len(final_cg)):
        key = ("_"+letters[i]) # Formatting for GRiTS into dictionary with a letter as the key
        grits_string[key] = final_cg[i] # SMIILES string of the backbone segment as the value
    system = CG_System(
        filename,
        beads=grits_string,
        conversion_dict=element_dict
    )
    cg_gsd = "CG-" + new_gsd_name + ".gsd"
    system.save(cg_gsd)
    return(grits_string)


In [ ]:
backbones = cg(path + "gsd/10_mers/PCPDTFBT_C4_BO_10mer_linear_test.gsd")

In [ ]:
gsdfile = path + "alternate_PCPDTPT_nC16_melt.gsd"
system = CG_System(
    gsdfile,
    beads={"_A": "C1SC2C3SC(C4C5=NSN=C5C=NC=4)=CC=3C(CCCCCCCCCCCCCCCC)(CCCCCCCCCCCCCCCC)C=2C=1"},
    conversion_dict=element_dict
)

In [4]:
temp = mb.load("C1SC2C3SC(C4C5=NSN=C5C=NC=4)=CC=3C(CCCCCCCCCCCCCCCC)(CCCCCCCCCCCCCCCC)C=2C=1",smiles=True)

In [5]:
gsdfile = path + "alternate_PCPDTPT_nC16_melt.gsd"
system = CG_System(
    gsdfile,
    beads={"_A": "CCCCCCCCCCCCCCCC"},
    conversion_dict=element_dict
)

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/grits/coarsegrain.py:194: UserWarning: Some atoms have been left out of coarse-graining!
  warn("Some atoms have been left out of coarse-graining!")


In [11]:
cg_gsd = "cg_test.gsd"
system.save(cg_gsd)

In [40]:
com_traj = gsd.hoomd.open("PCPDTFBT_C4_BO_10mer_linear_test.gsd")
coms = []
for frame in com_traj:
    particles = frame.particles
    pos,mass = utils.get_heavy_atoms(particles)
    coms.append(utils.get_com(pos,mass))

In [43]:
print(coms)

[array([-72.395935,  65.7702  ,  58.98571 ], dtype=float32), array([-72.395935,  65.7702  ,  58.9857  ], dtype=float32)]


In [7]:
# Appending a frame of a linear chain
original_traj = gsd.hoomd.open(path + "gsd_files/10_mers/PCPDTFBT_C4_BO_melt.gsd","r+")
new_traj = gsd.hoomd.open(path + "PCPDTFBT_C4_BO_10mer_linear.gsd","r+")
for frame in original_traj:
    new_traj.append(frame)

101
103
103
